# Querying

This notebook demonstrates Nexus Forge data [querying features](https://nexus-forge.readthedocs.io/en/latest/interaction.html#querying).

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource
from kgforge.specializations.resources import Dataset
from kgforge.core.wrappings.paths import Filter, FilterOperator

## Retrieval

### latest version

In [25]:
jane = Resource(type="Person", name="Jane Doe", award=["Nobel"])

In [26]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [27]:
resource = forge.retrieve(jane.id)

In [28]:
resource == jane

True

### specific version

In [162]:
jane = Resource(type="Person", name="Jane Doe", award=["Nobel"])

In [163]:
forge.register(jane)

<action> _register_one
<succeeded> True


In [164]:
forge.tag(jane, "v1")

<action> _tag_one
<succeeded> True


In [165]:
jane.email = ["jane.doe@epfl.ch", "jane.doe@example.org"]

In [166]:
forge.update(jane)

<action> _update_one
<succeeded> True


In [167]:
try:
    # DemoStore
    print(jane._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane._store_metadata._rev)

3


In [175]:
jane_v1 = forge.retrieve(jane.id, version=1)

In [176]:
jane_v1_tag = forge.retrieve(jane.id, version="v1")

In [178]:
jane_v1_rev = forge.retrieve(jane.id+"?rev=1")

In [179]:
jane_v1 == jane_v1_tag

True

In [180]:
jane_v1 == jane_v1_rev

True

In [181]:
jane_v1 != jane

True

In [39]:
try:
    # DemoStore
    print(jane_v1._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane_v1._store_metadata._rev)

1


### crossbucket retrieval
It is possible to retrieve resources stored in buckets different then the configured one. The configured store should of course support it.

In [40]:
resource = forge.retrieve(jane.id, cross_bucket=True) # cross_bucket defaults to False

In [41]:
resource._store_metadata

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d',
 '_constrainedBy': 'https://bluebrain.github.io/nexus/schemas/unconstrained.json',
 '_createdAt': '2022-02-09T11:55:06.667Z',
 '_createdBy': 'https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy',
 '_deprecated': False,
 '_incoming': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d/incoming',
 '_outgoing': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d/outgoing',
 '_project': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/mfsy',
 '_rev': 3,
 '_schemaProject': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/mfsy',
 '_self': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d',
 '_updatedAt': '2022-02-09T11:55:15.900Z',
 '_updatedBy': 'https://sandbox.bluebrainnexus.io/v1/realms

In [42]:
resource._last_action

Action(error=None, message=None, operation='retrieve', succeeded=True)

In [43]:
resource._synchronized

True

### Original source retrieval
When using BlueBrainNexusStore, it is possible to retrieve resources' payload as they were registered without any changes (retrieve_source=True).

In [44]:
forge._debug=True
resource = forge.retrieve(jane.id, retrieve_source=False) # retrieve_source defaults to True

In [45]:
forge.as_json(resource)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d',
 'type': 'Person',
 'award': 'Nobel',
 'email': ['jane.doe@epfl.ch', 'jane.doe@example.org'],
 'name': 'Jane Doe'}

In [46]:
resource._store_metadata

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d',
 '_constrainedBy': 'https://bluebrain.github.io/nexus/schemas/unconstrained.json',
 '_createdAt': '2022-02-09T11:55:06.667Z',
 '_createdBy': 'https://sandbox.bluebrainnexus.io/v1/realms/github/users/mfsy',
 '_deprecated': False,
 '_incoming': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d/incoming',
 '_outgoing': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d/outgoing',
 '_project': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/mfsy',
 '_rev': 3,
 '_schemaProject': 'https://sandbox.bluebrainnexus.io/v1/projects/github-users/mfsy',
 '_self': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/f1c5e23c-91fc-4449-a703-79d0cba35e3d',
 '_updatedAt': '2022-02-09T11:55:15.900Z',
 '_updatedBy': 'https://sandbox.bluebrainnexus.io/v1/realms

In [47]:
resource._last_action

Action(error=None, message=None, operation='retrieve', succeeded=True)

In [48]:
resource._synchronized

True

### error handling

In [51]:
resource = forge.retrieve("123")

<action> retrieve
<error> RetrievalError: 404 Client Error: Not Found for url: https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/%3A%2F%2F123/source



In [52]:
resource is None

True

## Searching

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel. Commented lines are for DemoModel.

In [53]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [54]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [55]:
dataset = Dataset(forge, type="Dataset", contribution=[contribution_jane, contribution_john])
dataset.add_distribution("../../data/associations.tsv")

In [56]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [57]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/dc2de106-9ae6-4721-aee2-758416420293',
 'type': 'Dataset',
 'contribution': [{'type': 'Contribution',
   'agent': {'type': 'Person', 'name': 'Jane Doe'}},
  {'type': 'Contribution', 'agent': {'type': 'Person', 'name': 'John Smith'}}],
 'distribution': {'type': 'DataDownload',
  'atLocation': {'type': 'Location',
   'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault'}},
  'contentSize': {'unitCode': 'bytes', 'value': 477},
  'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/9d02ced7-110a-4be6-8f8b-0b4df91251a7',
  'digest': {'algorithm': 'SHA-256',
   'value': '789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859'},
  'encodingFormat': 'text/tab-separated-values',
  'name': 'associations.tsv'}}

### Using resource paths as filters

The `paths` method load the template or property paths for a given type.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about templates and types.

In [58]:
p = forge.paths("Dataset")

You have autocompletion on `p` and this can be used to create search filters.

Note: There is a known issue for RdfModel which requires using `p.type.id` instead of `p.type`.

All [python comparison operators](https://www.w3schools.com/python/gloss_python_comparison_operators.asp) are supported.

In [59]:
resources = forge.search(p.type.id=="Person", limit=3)

In [60]:
type(resources)

list

In [61]:
len(resources)

3

In [62]:
forge.as_dataframe(resources)

,id,type,award,name,email
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,"[jane.doe@epfl.ch, jane.doe@example.org]"
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,NaN,Jane Doe,NaN


In [63]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,award,name,_constrainedBy,_createdAt,_createdBy,_deprecated,_incoming,_outgoing,_project,_rev,_schemaProject,_self,_updatedAt,_updatedBy,email
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-02-09T11:55:02.051Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-02-09T11:55:02.051Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-02-09T11:55:06.667Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,3,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-02-09T11:55:15.900Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,"[jane.doe@epfl.ch, jane.doe@example.org]"
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,NaN,Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-01-06T15:46:40.285Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-01-06T15:46:40.285Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,NaN


In [64]:
# Search results are not synchronized
resources[0]._synchronized

False

#### Using nested resource property

Property autocompletion is available on a path `p` even for nested properties like `p.contribution`.

In [65]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat
resources = forge.search(p.type.id == "Dataset", p.distribution.encodingFormat == "text/tab-separated-values", limit=3)

In [66]:
len(resources)

3

In [67]:
forge.as_dataframe(resources)

,id,type,contribution,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,506,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,9639abc864e91c645779f510ae5c06a1618941d569eb1a...,text/tab-separated-values,associations.tsv
1,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,506,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,9639abc864e91c645779f510ae5c06a1618941d569eb1a...,text/tab-separated-values,associations.tsv
2,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv


### Using dictionaries as filters
A dictionary can be provided for filters:
* {'type': {'id':'Dataset'}} is equivalent to p.type.id=="Dataset"
* only the '==' operator is supported
* nested dict are supported
* it is not mandatory for the provided properties and values to be defined in the forge model. Results will be retrieved if there are corresponding data in the store.

This feature is not supported when using the DemoStore


In [69]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat and created a given datetime.
filters = {"type": "Dataset", "distribution":{"encodingFormat":"text/tab-separated-values"}, "_createdAt":dataset._store_metadata._createdAt}
resources = forge.search(filters, limit=3)

In [70]:
type(resources)

list

In [71]:
len(resources)

1

In [72]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,contribution,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,...,_createdBy,_deprecated,_incoming,_outgoing,_project,_rev,_schemaProject,_self,_updatedAt,_updatedBy
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2022-02-09T12:46:03.048Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...


### Using built-in Filter objects

#### Supported filter operators

In [73]:
[f"{op.value} ({op.name})" for op in FilterOperator] # These are equivalent to the Python comparison operators

['__eq__ (EQUAL)',
 '__ne__ (NOT_EQUAL)',
 '__lt__ (LOWER_THAN)',
 '__le__ (LOWER_OR_Equal_Than)',
 '__gt__ (GREATER_Than)',
 '__ge__ (GREATER_OR_Equal_Than)']

In [74]:
# Search for resources of type Dataset and with text/tab-separated-values as distribution.encodingFormat and created before a given datetime.
filter_1 = Filter(operator=FilterOperator.EQUAL, path=["type"], value="Dataset")
filter_2 = Filter(operator=FilterOperator.EQUAL, path=["distribution","encodingFormat"], value="text/tab-separated-values")
filter_3 = Filter(operator=FilterOperator.LOWER_OR_Equal_Than, path=["_createdAt"], value=dataset._store_metadata._createdAt)

resources = forge.search(filter_1, filter_2, filter_3, limit=3)

In [75]:
type(resources)

list

In [76]:
len(resources)

3

In [77]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,contribution,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,...,_createdBy,_deprecated,_incoming,_outgoing,_project,_rev,_schemaProject,_self,_updatedAt,_updatedBy
0,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,506,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2021-08-17T12:29:43.889Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...
1,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,506,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2021-08-17T12:33:52.097Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...
2,https://sandbox.bluebrainnexus.io/v1/resources...,Dataset,"[{'type': 'Contribution', 'agent': {'type': 'P...",DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,477,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,...,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/projects/...,1,https://sandbox.bluebrainnexus.io/v1/projects/...,https://sandbox.bluebrainnexus.io/v1/resources...,2021-08-23T09:31:11.771Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...


### Using search endpoints

Two types of search endpoints are supported: 'sparql' (default) for graph queries and 'elastic' for document oriented queries. The types of available search endpoint can be configured (see [00-Initialization.ipynb](00%20-%20Initialization.ipynb) for an example of search endpoints config) or set when creating a KnowledgeGraphForge session using the 'searchendpoints' arguments.

The search endpoint to hit when calling forge.search(...) is 'sparql' by default but can be specified using the 'search_endpoint' argument.

#### SPARQL Search Endpoint

In [78]:
# Search for resources of type Person
filters = {"type": "Person"}
resources = forge.search(filters, limit=3, search_endpoint='sparql')

In [79]:
type(resources)

list

In [80]:
len(resources)

3

In [81]:
forge.as_dataframe(resources, store_metadata=True)

,id,type,award,name,_constrainedBy,_createdAt,_createdBy,_deprecated,_incoming,_outgoing,...,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-02-09T11:55:02.051Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://sandbox.bluebrainnexus.io/v1/resources...,Person,[Nobel],Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2022-02-09T11:55:06.667Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://sandbox.bluebrainnexus.io/v1/resources...,Person,NaN,Jane Doe,https://bluebrain.github.io/nexus/schemas/unco...,2021-09-22T16:33:24.690Z,https://sandbox.bluebrainnexus.io/v1/realms/gi...,False,https://sandbox.bluebrainnexus.io/v1/resources...,https://sandbox.bluebrainnexus.io/v1/resources...,...,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,477.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,789aa07948683fe036ac29811814a826b703b562f7d168...,text/tab-separated-values,associations.tsv


#### ElasticSearch Endpoint

In [82]:
# Search for resources of type Person and retrieve their ids and names.
filters = {"@type": "http://schema.org/Person"}
resources = forge.search(filters, limit=3, search_endpoint='elastic', includes=["@id","name"]) # fields can also be excluded with 'excludes'

In [83]:
type(resources)

list

In [84]:
len(resources)

3

In [85]:
forge.as_dataframe(resources, store_metadata=True)

,@id,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe
1,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe
2,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe


In [ ]:
# Search results are not synchronized
resources[0]._synchronized

In [ ]:
resources[0].id

In [ ]:
resources[0].type

### Crossbucket search
It is possible to search for resources stored in buckets different than the configured one. The configured store should of course support it.

In [87]:
resources = forge.search(p.type.id == "Association", limit=3, cross_bucket=True)  # cross_bucket defaults to False

In [88]:
type(resources)

list

In [89]:
len(resources)

3

In [90]:
forge.as_dataframe(resources)

,id,type,agent.type,agent.gender.id,agent.gender.type,agent.gender.label,agent.name,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name,name
0,https://kg.example.ch/associations/123,Association,Person,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,Marie Curie,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,46.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,e0fe65f725bf28fe2b88c7bafb51fb5ef1df0ab14c68a3...,text/plain,marie_curie.txt,Curie Association
1,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,Albert Einstein,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,50.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,91a5ce5c84dc5bead730a4b49d0698b4aaef4bc06ce164...,text/plain,albert_einstein.txt,Einstein Association
2,https://sandbox.bluebrainnexus.io/v1/resources...,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Furthermore it is possible to filter by bucket when cross_bucket is set to True. Setting a bucket value when cross_bucket is False will trigger a not_supported exception.
resources = forge.search(p.type.id == "Person", limit=3, cross_bucket=True, bucket=<str>)  # add a bucket

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

### Searching original source
When using BlueBrainNexusStore, it is possible to retrieve resources' payload as they were registered without any changes (retrieve_source=True).

In [92]:
resources = forge.search(p.type.id == "Association", limit=3, retrieve_source=False)  # retrieve_source defaults to True

In [93]:
type(resources)

list

In [94]:
len(resources)

3

In [95]:
forge.as_dataframe(resources)

,id,type,agent.type,agent.gender.id,agent.gender.type,agent.gender.label,agent.name,distribution.type,distribution.atLocation.type,distribution.atLocation.store.id,distribution.contentSize.unitCode,distribution.contentSize.value,distribution.contentUrl,distribution.digest.algorithm,distribution.digest.value,distribution.encodingFormat,distribution.name,name
0,https://kg.example.ch/associations/123,Association,Person,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,Marie Curie,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,46.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,e0fe65f725bf28fe2b88c7bafb51fb5ef1df0ab14c68a3...,text/plain,marie_curie.txt,Curie Association
1,NaN,Association,Person,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,Albert Einstein,DataDownload,Location,https://bluebrain.github.io/nexus/vocabulary/d...,bytes,50.0,https://sandbox.bluebrainnexus.io/v1/files/git...,SHA-256,91a5ce5c84dc5bead730a4b49d0698b4aaef4bc06ce164...,text/plain,albert_einstein.txt,Einstein Association
2,NaN,Association,Person,NaN,NaN,NaN,Jane Doe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Graph traversing

SPARQL is used as a query language to perform graph traversing.

Nexus Forge implements a SPARQL query rewriting strategy leveraging a configured RDFModel that lets users write SPARQL queries without adding prefix declarations, prefix names or long IRIs. With this strategy, only type and property names can be provided.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about templates.

Note: DemoStore doesn't implement SPARQL operations yet. Please use another store for this section.

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel.

In [96]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [97]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [98]:
association = Resource(type="Dataset", contribution=[contribution_jane, contribution_john])

In [99]:
forge.register(association)

<action> _register_one
<succeeded> True


In [102]:
forge.template("Dataset") # Templates help know which property to use when writing a query to serach for a given type

{
    id: ""
    type:
    {
        id: ""
    }
    annotation:
    {
        id: ""
        type: Annotation
        hasBody:
        {
            id: ""
            type:
            {
                id: ""
            }
            label: ""
            note: ""
        }
        hasTarget:
        {
            id: ""
            type: AnnotationTarget
        }
        note: ""
    }
    brainLocation:
    {
        id: ""
        type: BrainLocation
        atlasSpatialReferenceSystem:
        {
            id: ""
            type: AtlasSpatialReferenceSystem
        }
        brainRegion:
        {
            id: ""
            label: ""
        }
        coordinatesInBrainAtlas:
        {
            id: ""
            valueX: 0.0
            valueY: 0.0
            valueZ: 0.0
        }
        coordinatesInSlice:
        {
            spatialReferenceSystem:
            {
                id: ""
                type: SpatialReferenceSystem
            }
            valueX

### Prefix and namespace free SPARQL query

When a forge RDFModel is configured, then there is no need to provide prefixes and namespaces when writing a SPARQL query. Prefixes and namespaces will be automatically inferred from the provided schemas and/or JSON-LD context and the query rewritten accordingly.

In [103]:
query = """
    SELECT ?id ?name
    WHERE {
        ?id a Dataset ;
        contribution/agent ?contributor.
        ?contributor name ?name.
    }
"""

In [104]:
resources = forge.sparql(query, limit=3)

In [105]:
type(resources)

list

In [106]:
len(resources)

3

In [107]:
type(resources[0])

kgforge.core.resource.Resource

In [108]:
forge.as_dataframe(resources)

,id,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe
1,https://sandbox.bluebrainnexus.io/v1/resources...,John Smith
2,https://sandbox.bluebrainnexus.io/v1/resources...,John Smith


### display rewritten SPARQL query 

In [109]:
resources = forge.sparql(query, limit=3, debug=True)

Submitted query:
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   
       SELECT ?id ?name
       WHERE {
           ?id a schema:Dataset ;
           nsg:contribution/prov:agent ?contributor.
           ?contributor sc

### Full SPARQL query

Regular SPARQL query can also be provided. When provided, the limit and offset arguments superseed any in query limit or offset values.

In [131]:
query = """
PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   SELECT ?id ?name
   WHERE {
       ?id a schema:Dataset ;
       nsg:contribution/prov:agent ?contributor.
       ?contributor schema:name ?name.
   }
   ORDER BY ?id
   LIMIT 1
   OFFSET 0
"""

In [132]:
# it is recommended to set 'rewrite' to 'False' to prevent the sparql query rewriting when a syntactically correct SPARQL query is provided.
resources = forge.sparql(query, rewrite=False, limit=3, offset=1, debug=True) 

Submitted query:
   
   PREFIX dc: <http://purl.org/dc/elements/1.1/>
      PREFIX dcat: <http://www.w3.org/ns/dcat#>
      PREFIX dcterms: <http://purl.org/dc/terms/>
      PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
      PREFIX nsg: <https://neuroshapes.org/>
      PREFIX owl: <http://www.w3.org/2002/07/owl#>
      PREFIX prov: <http://www.w3.org/ns/prov#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      PREFIX schema: <http://schema.org/>
      PREFIX sh: <http://www.w3.org/ns/shacl#>
      PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
      PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
      PREFIX vann: <http://purl.org/vocab/vann/>
      PREFIX void: <http://rdfs.org/ns/void#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      PREFIX : <https://neuroshapes.org/>
      SELECT ?id ?name
      WHERE {
          ?id a schema:Dataset ;
          nsg:contribution/prov:ag

In [133]:
type(resources)

list

In [134]:
len(resources)

3

In [135]:
type(resources[0])

kgforge.core.resource.Resource

In [136]:
forge.as_dataframe(resources)

,id,name
0,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe
1,https://sandbox.bluebrainnexus.io/v1/resources...,Jane Doe
2,https://sandbox.bluebrainnexus.io/v1/resources...,John Smith


## ElasticSearch DSL Query

ElasticSearch DSL can be used as a query language search for resources provided that the configured store supports it. The 'BlueBrainNexusStore' supports ElasticSearch.

Note: DemoStore doesn't implement ElasaticSearch DSL operations.

In [137]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [138]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [139]:
association = Resource(type="Dataset", contribution=[contribution_jane, contribution_john])

In [140]:
forge.register(association)

<action> _register_one
<succeeded> True


In [142]:
forge.template("Dataset") # Templates help know which property to use when writing a query  to search for a given type

{
    id: ""
    type:
    {
        id: ""
    }
    annotation:
    {
        id: ""
        type: Annotation
        hasBody:
        {
            id: ""
            type:
            {
                id: ""
            }
            label: ""
            note: ""
        }
        hasTarget:
        {
            id: ""
            type: AnnotationTarget
        }
        note: ""
    }
    brainLocation:
    {
        id: ""
        type: BrainLocation
        atlasSpatialReferenceSystem:
        {
            id: ""
            type: AtlasSpatialReferenceSystem
        }
        brainRegion:
        {
            id: ""
            label: ""
        }
        coordinatesInBrainAtlas:
        {
            id: ""
            valueX: 0.0
            valueY: 0.0
            valueZ: 0.0
        }
        coordinatesInSlice:
        {
            spatialReferenceSystem:
            {
                id: ""
                type: SpatialReferenceSystem
            }
            valueX

### Plain ElasticSearch DSL

In [143]:
query = """
        {
          "_source": {
            "includes": [
              "@id",
              "name"
            ]
          },
          "query": {
            "term": {
              "@type": "http://schema.org/Dataset"
            }
          }
        }
"""

In [144]:
resources = forge.elastic(query, limit=3) # limit and offset (when provided in this method call) superseed 'size' and 'from' values provided in the query

In [145]:
type(resources)

list

In [146]:
len(resources)

3

In [147]:
type(resources[0])

kgforge.core.resource.Resource

In [148]:
forge.as_dataframe(resources)

,@id,name
0,https://bbp.epfl.ch/neurosciencegraph/data/neu...,Scnn1a-Tg3-Cre;Ai14-187849.06.01.01
1,https://bbp.epfl.ch/neurosciencegraph/data/neu...,H17.06.004.11.05.04
2,https://bbp.epfl.ch/neurosciencegraph/data/neu...,H16.06.009.01.01.15.01


## Downloading

Note: DemoStore doesn't implement file operations yet. Please use another store for this section.

In [149]:
jane = Resource(type="Person", name="Jane Doe")

In [150]:
! ls -p ../../data | egrep -v /$

associations.tsv
my_data.xwz
my_data_derived.txt
persons.csv
tfidfvectorizer_model_schemaorg_linking


In [151]:
distribution = forge.attach("../../data")

In [152]:
association = Resource(type="Association", agent=jane, distribution=distribution)

In [153]:
forge.register(association)

<action> _register_one
<succeeded> True


In [158]:
# The argument overwrite: bool can be provided to decide whether to overwrite (True) existing files with the same name or
# to create new ones (False) with their names suffixed with a timestamp.
# A cross_bucket argument can be provided to download data from the configured bucket (cross_bucket=False - the default value) 
# or from a bucket different than the configured one (cross_bucket=True). The configured store should support crossing buckets for this to work.
forge.download(association, "distribution.contentUrl", "./downloaded/")

In [159]:
! ls -l ./downloaded/

total 440
-rw-r--r--  1 mfsy  staff     477 Feb  9 13:57 associations.tsv
-rw-r--r--  1 mfsy  staff      16 Feb  9 13:57 my_data.xwz
-rw-r--r--  1 mfsy  staff      24 Feb  9 13:57 my_data_derived.txt
-rw-r--r--  1 mfsy  staff      52 Feb  9 13:57 persons.csv
-rw-r--r--  1 mfsy  staff  204848 Feb  9 13:57 tfidfvectorizer_model_schemaorg_linking


In [160]:
#! rm -R ./downloaded/